In [36]:
#!pip install pyspark


#need to save: books with titles, songs, books with genres, reviews table




In [1]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, struct, array_except, size, length, map_keys, from_json, array_intersect, split  # import struct for nested field renaming

In [ ]:
NAMENODE = "localhost"
PORT     = "9000"
HDFS     = f"hdfs://{NAMENODE}:{PORT}"
BRONZE   = f"{HDFS}/bronze"
SILVER   = f"{HDFS}/silver"

# Ініціалізація SparkSession
spark = SparkSession.builder \
    .appName("SilverLayerETL_Debug") \
    .config("spark.hadoop.fs.defaultFS", HDFS) \
    .config("spark.driver.memory", "8g")\
    .getOrCreate()

'spark = SparkSession.builder     .appName("SilverLayerETL_Debug")     .config("spark.hadoop.fs.defaultFS", HDFS)     .config("spark.driver.memory", "8g")    .getOrCreate()'

In [3]:

# Функція перевірки існування шляху в HDFS
def check_path(path):
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(
        spark._jsc.hadoopConfiguration())
    p  = spark._jvm.org.apache.hadoop.fs.Path(path)
    ok = fs.exists(p)
    print(f"DEBUG: Path {path} exists: {ok}")
    if not ok:
        sys.exit(1)

In [5]:
songs_path     = f"{BRONZE}/light_spotify_dataset.csv"
check_path(songs_path)

songs_raw = spark.read.option("header", True).option("inferSchema", True) \
               .csv(songs_path)
print(f"Loaded songs_raw columns: {songs_raw.columns}, rows: {songs_raw.count()}")
songs_raw.show(5, truncate=False)

songs_dropped=songs_raw.drop('Release Date', 'Explicit', "Key", "Popularity")
songs_clean = songs_dropped.filter(
        col("artist").isNotNull() & (length(col("artist")) > 1) &
        col("song").isNotNull() & (length(col("song")) > 1) &
        col("emotion").isNotNull() & (length(col("emotion")) > 1) &

        (col("variance") <1) & (col("variance") >-1) &

        (col("Tempo") > 0) & (col("Tempo") <= 300) &

        (col("Loudness") >= -60) & (col("Loudness") <= 0) &

        (col("Energy") >= 0) & (col("Energy") <= 100) &
        (col("Danceability") >= 0) & (col("Danceability") <= 100) &
        (col("Positiveness") >= 0) & (col("Positiveness") <= 100) &
        (col("Speechiness") >= 0) & (col("Speechiness") <= 100) &
        (col("Liveness") >= 0) & (col("Liveness") <= 100) &
        (col("Acousticness") >= 0) & (col("Acousticness") <= 100) &
        (col("Instrumentalness") >= 0) & (col("Instrumentalness") <= 100) 
    )
songs_clean.show(10, truncate=False)

DEBUG: Path hdfs://localhost:9000/bronze/light_spotify_dataset.csv exists: True


Loaded songs_raw columns: ['artist', 'song', 'emotion', 'variance', 'Genre', 'Release Date', 'Key', 'Tempo', 'Loudness', 'Explicit', 'Popularity', 'Energy', 'Danceability', 'Positiveness', 'Speechiness', 'Liveness', 'Acousticness', 'Instrumentalness'], rows: 236988
+------+---------------------+-------+-------------------+-----+------------+------+-----+--------+--------+----------+------+------------+------------+-----------+--------+------------+----------------+
|artist|song                 |emotion|variance           |Genre|Release Date|Key   |Tempo|Loudness|Explicit|Popularity|Energy|Danceability|Positiveness|Speechiness|Liveness|Acousticness|Instrumentalness|
+------+---------------------+-------+-------------------+-----+------------+------+-----+--------+--------+----------+------+------------+------------+-----------+--------+------------+----------------+
|ABBA  |She's My Kind Of Girl|joy    |0.4476190476190476 |pop  |2014        |F Maj |128  |-6.0    |No      |31        |78 

In [9]:
songs_clean.write.mode("overwrite").parquet(f'{SILVER}/song_info')

In [10]:
from pyspark.sql.functions import when, lit, array, array_remove, col, expr, array_compact

# books genres
books_path = f"{BRONZE}/goodreads_book_genres_initial.json"
check_path(books_path)
books_raw = spark.read.json(books_path)
books_raw.printSchema()

books_genres=books_raw.select(
    "book_id",
    col("genres.children").alias("children"),
    col("genres.`comics, graphic`").alias("comics_graphic"),
    col("genres.`fantasy, paranormal`").alias("fantasy_paranormal"),
    col("genres.fiction").alias("fiction"),
    col("genres.`history, historical fiction, biography`").alias("history_biography"),
    col("genres.`mystery, thriller, crime`").alias("mystery_thriller_crime"),
    col("genres.`non-fiction`").alias("non_fiction"),
    col("genres.poetry").alias("poetry"),
    col("genres.romance").alias("romance"),
    col("genres.`young-adult`").alias("young_adult")
)
field_names = books_raw.schema["genres"].dataType.names
books_genres = books_raw.select(col('book_id'),array([*[when(col(f"genres.{name}").isNotNull(), lit(name)) for name in field_names]]).alias("keys"))
books_genres = books_genres.withColumn(
    "genres",
    array_compact("keys")
).drop('keys')
books_genres.show(10,truncate=False)
print(books_raw.count)

DEBUG: Path hdfs://localhost:9000/bronze/goodreads_book_genres_initial.json exists: True


root
 |-- book_id: string (nullable = true)
 |-- genres: struct (nullable = true)
 |    |-- children: long (nullable = true)
 |    |-- comics, graphic: long (nullable = true)
 |    |-- fantasy, paranormal: long (nullable = true)
 |    |-- fiction: long (nullable = true)
 |    |-- history, historical fiction, biography: long (nullable = true)
 |    |-- mystery, thriller, crime: long (nullable = true)
 |    |-- non-fiction: long (nullable = true)
 |    |-- poetry: long (nullable = true)
 |    |-- romance: long (nullable = true)
 |    |-- young-adult: long (nullable = true)

+--------+---------------------------------------------------------------------------------------------+
|book_id |genres                                                                                       |
+--------+---------------------------------------------------------------------------------------------+
|5333265 |[history, historical fiction, biography]                                                     |
|

In [11]:
books_with_titles_path = f"hdfs://localhost:9000/bronze/goodreads_books.json"
check_path(books_with_titles_path)
books_with_titles_raw = spark.read.json(books_with_titles_path)
books_with_titles_raw.printSchema()
books_with_titles_clean=books_with_titles_raw.select(
    col("book_id"),
    col("title"),
    col("isbn"),
    col("description")
)

books_genres_total_raw=books_with_titles_clean.join(books_genres, 'book_id')


DEBUG: Path hdfs://localhost:9000/bronze/goodreads_books.json exists: True


root
 |-- asin: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- role: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- format: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- link: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- popular_shelves: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- count: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- pub

In [12]:
books_genres_total_raw.printSchema()
books_genres_total_raw.show()

root
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- description: string (nullable = true)
 |-- genres: array (nullable = false)
 |    |-- element: string (containsNull = true)



+--------+--------------------+----------+--------------------+--------------------+
| book_id|               title|      isbn|         description|              genres|
+--------+--------------------+----------+--------------------+--------------------+
|10000600|Old Habits (Wicke...|          |                    |[fantasy, paranor...|
| 1000073|                Home|0571228437|When their tradit...|           [fiction]|
|10000761|Render Unto Rome:...|038553132X|AN INVESTIGATION ...|[history, histori...|
| 1000096|Strontium Dog: Se...|1905437153|Earth, the late 2...|[comics, graphic,...|
|10001249|Revelation (Matth...|1440743975|Spring, 1543. Kin...|[fiction, history...|
|10001298|Fall with Honor: ...|1441808477|Freedom is on the...|[comics, graphic,...|
|10001383|              33 Men|0399157778|Having had unpara...|[history, histori...|
|10001412|The Hazards of th...|145023884X|Life ends, new li...|[fantasy, paranor...|
| 1000146|A Theology for th...|080542640X|A Theology for th...|[h

In [12]:
from pyspark.sql.functions import explode, col

books_exploded = books_genres_total_raw.select(
    col("book_id"),
    explode(col("genres")).alias("genre")
)

# Split genres that are comma-separated into individual rows
books_flat = books_exploded.select(
    col("book_id"),
    explode(split(col("genre"), ",\\s*")).alias("genre")
)

books_flat.show(truncate=False)

##save this to silver layer

+--------+------------------+
|book_id |genre             |
+--------+------------------+
|10000273|non-fiction       |
|10000578|history           |
|10000578|historical fiction|
|10000578|biography         |
|10000600|fantasy           |
|10000600|paranormal        |
|10000600|fiction           |
|10000600|romance           |
|10000600|young-adult       |
|1000073 |fiction           |
|10000761|history           |
|10000761|historical fiction|
|10000761|biography         |
|10000761|mystery           |
|10000761|thriller          |
|10000761|crime             |
|10000761|non-fiction       |
|1000087 |fantasy           |
|1000087 |paranormal        |
|1000087 |fiction           |
+--------+------------------+
only showing top 20 rows



In [25]:
books_flat.write.mode("overwrite").parquet(f'{SILVER}/books_genres_exploded')

In [14]:
books_total_raw=books_genres_total_raw.drop('genres')
books_total_raw.show(10)

##save this to silver


+--------+--------------------+----------+--------------------+
| book_id|               title|      isbn|         description|
+--------+--------------------+----------+--------------------+
|10000600|Old Habits (Wicke...|          |                    |
| 1000073|                Home|0571228437|When their tradit...|
|10000761|Render Unto Rome:...|038553132X|AN INVESTIGATION ...|
| 1000087|Trojan Odyssey (D...|0718147022|A daring rescue s...|
| 1000096|Strontium Dog: Se...|1905437153|Earth, the late 2...|
|10001249|Revelation (Matth...|1440743975|Spring, 1543. Kin...|
| 1000127|Double Play at Short|0316142018|Twelve-year-old D...|
|10001298|Fall with Honor: ...|1441808477|Freedom is on the...|
|10001383|              33 Men|0399157778|Having had unpara...|
|10001412|The Hazards of th...|145023884X|Life ends, new li...|
+--------+--------------------+----------+--------------------+
only showing top 10 rows



In [19]:

books_total_raw.write.mode("overwrite").parquet(f'{SILVER}/books_with_titles')

In [16]:
review_path = f"hdfs://localhost:9000/bronze/goodreads_reviews_dedup.json"
check_path(review_path)
rewiew_raw = spark.read.json(review_path).select(col('book_id'), col("review_text"), col("user_id"))
rewiew_raw.printSchema()
rewiew_raw.show(10)

DEBUG: Path hdfs://localhost:9000/bronze/goodreads_reviews_dedup.json exists: True


root
 |-- book_id: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- user_id: string (nullable = true)

+--------+--------------------+--------------------+
| book_id|         review_text|             user_id|
+--------+--------------------+--------------------+
|24375664|Mind blowingly co...|8842281e1d1347389...|
|18245960|This is a special...|8842281e1d1347389...|
| 6392944|I haven't read a ...|8842281e1d1347389...|
|22078596|Fun, fast paced, ...|8842281e1d1347389...|
| 6644782|A fun book that g...|8842281e1d1347389...|
| 1995421|Kevin highly reco...|8842281e1d1347389...|
| 9460786|Giving a high rat...|8842281e1d1347389...|
|29983426|I decided to give...|8842281e1d1347389...|
|29893493|I haven't read a ...|8842281e1d1347389...|
|28114110|Kevin Kelly, who ...|8842281e1d1347389...|
+--------+--------------------+--------------------+
only showing top 10 rows



In [18]:
rewiew_raw.write.mode("overwrite").parquet(f'{SILVER}/rewiew_info')

In [21]:
df = spark.read.parquet(f'{SILVER}/song_info')
df.show()
df.printSchema()
print(df.count())

+------+--------------------+--------+--------------------+-----+-----+--------+------+------------+------------+-----------+--------+------------+----------------+
|artist|                song| emotion|            variance|Genre|Tempo|Loudness|Energy|Danceability|Positiveness|Speechiness|Liveness|Acousticness|Instrumentalness|
+------+--------------------+--------+--------------------+-----+-----+--------+------+------------+------------+-----------+--------+------------+----------------+
|  ABBA|She's My Kind Of ...|     joy|  0.4476190476190476|  pop|  128|    -6.0|    78|          56|          60|          3|      31|           7|               0|
|  ABBA|    Andante, Andante|    love| 0.20222222222222216|  pop|  102|  -10.72|    36|          52|          38|          2|       7|          68|               0|
|  ABBA|      As Good As New| sadness|  0.3008807588075881|  pop|  139|    -5.7|    78|          85|          97|          3|       8|          20|               2|
|  ABBA|  

In [34]:
df = spark.read.parquet(f'{SILVER}/rewiew_info')
df.orderBy(col("book_id")).show()
df.printSchema()
print(df.count())

+-------+--------------------+--------------------+
|book_id|         review_text|             user_id|
+-------+--------------------+--------------------+
|      1|** spoiler alert ...|fd5b6c438740d10ff...|
|      1|It was a cry fest...|b99819566982304d6...|
|      1|I knew what was g...|76f5e8e2d90996887...|
|      1|This book is anot...|c10742318a55edb61...|
|      1|** spoiler alert ...|527e350993da87abb...|
|      1|gaK gHettu baGus....|cbf421bfb41c30e73...|
|      1|I can't believe S...|8268861943409d7f3...|
|      1|Why did he have t...|7725374d6eef963ad...|
|      1|** spoiler alert ...|8a32aee1eb5a643ac...|
|      1|Wow, every time y...|9ed4e634a9230df7e...|
|      1|         Wonderful..|48120fdd3c46c53e1...|
|      1|I loved the inter...|f074d8b42ccf888a0...|
|      1|Siempre es doloro...|56ae5f1d46e27df61...|
|      1|(Reread June 2016...|a8f8dd7892906b084...|
|      1|I actually read t...|284c2159acb71167c...|
|      1|The "Half-Blood P...|ffddeb790b49eb8e1...|
|      1|I f

In [32]:
df = spark.read.parquet(f'{SILVER}/books_genres_exploded')
df.show()
df.printSchema()
print(df.count())

+--------+------------------+
| book_id|             genre|
+--------+------------------+
|       1|          children|
|       1|           fantasy|
|       1|        paranormal|
|       1|           fiction|
|       1|           mystery|
|       1|          thriller|
|       1|             crime|
|       1|           romance|
|       1|       young-adult|
|10000053|           fiction|
|10000053|           history|
|10000053|historical fiction|
|10000053|         biography|
|10000053|           romance|
|10000112|           fantasy|
|10000112|        paranormal|
|10000112|           fiction|
|10000112|           romance|
|10000236|           fantasy|
|10000236|        paranormal|
+--------+------------------+
only showing top 20 rows

root
 |-- book_id: string (nullable = true)
 |-- genre: string (nullable = true)

8126034


In [35]:
df = spark.read.parquet(f'{SILVER}/books_with_titles')
df.orderBy(col("book_id")).show()
df.printSchema()
print(df.count())

+--------+--------------------+----------+--------------------+
| book_id|               title|      isbn|         description|
+--------+--------------------+----------+--------------------+
|       1|Harry Potter and ...|0439785960|The war against V...|
|      10|Harry Potter Coll...|0439827604|Six years of magi...|
|     100|Simply Beautiful ...|1581805632|Blend your creati...|
|    1000|Millionaire Women...|0740745328|"Most Americans a...|
|   10000| The Face of Another|0375726535|Like an elegantly...|
|10000006|              Hatch!|0761458824|                    |
|10000009|How to Teach a Sl...|0761458050|A little boy expl...|
|10000014|           Pond Walk|0761458166|Buddy Bear and Ma...|
|10000034|Princess in Train...|1400098777|It's an all new s...|
|10000045|Princess in Train...|1417730315|Princess for pres...|
| 1000005|         Larky Mavis|0374343659|Another orginal p...|
|10000053|    Old Saint Paul's|1598188135|THE PESTILENCE To...|
| 1000006|Computers, Chess ...|038790012

In [ ]:
spark.stop()